# Simulation and plotting

In [ ]:
%matplotlib notebook
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import h5py
from model.DEM import DeepEuler
from model.Euler import Euler
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 12.0})

In [ ]:
#Load results from text files, if C++ solvers were used
#dem = np.loadtxt('simulations/vdp_dem_0.1_43.txt')
#euler = np.loadtxt('simulations/vdp_euler_0.1.txt')

In [ ]:
mu = 1.5
def vdp( t, x):
    y = np.empty(x.shape)
    y[0] = -mu*(x[1]*x[1]-1)*x[0]-x[1]
    y[1] = x[0]
    return y

## Integrate the ODE

In [ ]:
sol = scipy.integrate.solve_ivp(vdp, [0, 50], [ 1.0, 1.0], rtol=1e-10, atol=1e-10)

In [ ]:
eul_sol = scipy.integrate.solve_ivp(vdp, [0, 50], [ 1.0, 1.0], method=Euler, h=2**(-4))

Change the *traced_model_path* and *scaler_path* to point to your trained model and its scaler

In [ ]:
dem_sol = scipy.integrate.solve_ivp(vdp, [0, 50], [ 1.0, 1.0], method=DeepEuler, mode=DeepEuler.MODE_AUTONOMOUS, h=2**(-4),
                                    traced_model_path="training/traced_model_vdp_final_e589_2109290802.pt", 
                                    scaler_path="training/scaler_vdp_final_2109290802.psca")

## Plots

In [ ]:
plt.figure(num="PhaseSpace",figsize=(8,6))
plt.scatter(sol.y[1,:],sol.y[0,:], s=10, label="Dopri")
#plt.scatter(euler[:250,1],euler[:250,2],s=10, label="Euler")
plt.scatter(eul_sol.y[1,:],eul_sol.y[0,:], s=10, label="Euler")
plt.scatter(dem_sol.y[1,:],dem_sol.y[0,:], s=10, label="DEM")
#plt.scatter(dem[:,1],dem[:,2],s=10, label="DEM C++")
plt.xlabel("$x_2$")
plt.ylabel("$x_1$")
plt.legend()
plt.show()

In [ ]:
plt.figure(num="Comparison")
plt.plot(sol.t,sol.y[0,:], color="black", label="Dopri")
#plt.plot(euler[:,0],euler[:,1], color="silver", label="Euler")
plt.plot(eul_sol.t,eul_sol.y[0,:], color="purple", label="Euler")
#plt.plot(dem[:,0],dem[:,1], linestyle="--", dashes=(5,5), color="cyan", label="DEM C++")
plt.plot(dem_sol.t,dem_sol.y[0,:], linestyle="--", dashes=(5,5), color="orange", label="DEM")
plt.xlabel("$t$")
plt.ylabel("$x_1$")
plt.xlim([0, 40])
plt.ylim([-4,4])
plt.legend()
plt.show()